In [1]:
import numpy as np 
from scipy.optimize import fsolve

## Adiabatic model

In [2]:
R = 8.314
pi = np.pi
f_prime_guess = 0.02


In [51]:
P1 = 30*100000 #Pa
T1 = 25+273.15
MM = 2.016
mu1 = 0.00899 #Cp
W_per_sec = 1.97637 #kg/sec
W = W_per_sec*3600 #kg/h
D1 = 0.1397
L1 = 2 #m Just a guess
e = 0.000046 #m
Cp1 = 14.3346
Cv1 = 14.3346/1.42785

In [52]:
A1 = pi*D1**2/4

rho1 = MM*P1/(R*T1)/1000

u1 = W_per_sec/(rho1*A1)

print(A1, u1, rho1)

0.015327901242699303 52.846789760909175 2.439871469442849


In [53]:
def f_prime(x,e,D1,Re1):
    LHS=1/np.sqrt(x)
    RHS=-2*np.log10(e/D1/3.7+2.51/Re1/np.sqrt(x))
    return LHS - RHS

In [54]:
def rho(rho2, f_prime, L1, D1, k, rho1, P1, A1, W):
    LHS = f_prime*L1/D1
    RHS = ((k+1)/k)*np.log(rho2/rho1) + (1-(rho2/rho1)**2)*((k-1)/(2*k)+P1*rho1*(A1/W)**2)
    return LHS - RHS

In [55]:
def P(P2, W, A1, k, P1, rho1, rho2):
    LHS = (W/(rho1*A1))**2 + 2*k*(P1/rho1)/(k-1)
    RHS = (W/(rho2*A1))**2 + 2*k*(P2/rho2)/(k-1)
    return LHS - RHS

In [56]:
k = Cp1/Cv1

Re1 = rho1*u1*D1/(mu1)

f_prime_1 = fsolve(f_prime, f_prime_guess, args=(e,D1,Re1))[0]

rho2 = fsolve(rho, rho1, args=(f_prime_1, L1, D1, k, rho1, P1, A1, W_per_sec))[0]

P2 = fsolve(P,P1,args=(W_per_sec, A1, k, P1, rho1, rho2))[0]

print(rho1)
print(f_prime_1)
print(rho2)
print(P2/100000)

2.439871469442849
0.04967410221591388
2.4378969958539827
29.975705876506854


### Sonic velocity

In [57]:
c = np.sqrt(k*P2/rho1)
print(c)

1324.4712268392348


### Calculation of thicknesses

In [58]:
#-----------------------------------Known values-----------------------------------------------
M = 1.125
S = 110.32e6
E = 1
Y = 0.4

In [59]:
def calc_tmin(M,S,E,Y,P,d,C):
    return M*((P*d/(2*(S*E+P*Y)))+C)

In [60]:
tmin = calc_tmin(M,S,E,Y,P2,D1*1000,2)
print("The minimum wall thickness is {} mm".format(np.round(tmin,2)))

The minimum wall thickness is 4.36 mm


In [61]:
t = 0.006   #m
D1_rev = D1 - 2*t
print(D1, D1_rev)

0.1397 0.12769999999999998


Redo calculations with new diameter

In [62]:
A1 = pi*D1_rev**2/4

rho1 = MM*P1/(R*T1)/1000

u1 = W_per_sec/(rho1*A1)

print(A1, u1, rho1)

0.012807715615989571 63.24549972343547 2.439871469442849


In [50]:
Re1 = rho1*u1*D1_rev/(mu1)

f_prime_1 = fsolve(f_prime, f_prime_guess, args=(e,D1_rev,Re1))[0]

rho2 = fsolve(rho, rho1, args=(f_prime_1, L1, D1_rev, k, rho1, P1, A1, W_per_sec))[0]

P2 = fsolve(P,P1,args=(W_per_sec, A1, k, P1, rho1, rho2))[0]

print(rho1)
print(f_prime_1)
print(rho2)
print(P2/100000)

2.439871469442849
0.048299680167579934
2.4323373821314154
29.90727245699999


### Isothermal model